### [Pensador](https://www.pensador.com/)

#### Libraries

In [1]:
!pip install -U python-dotenv

Requirement already up-to-date: python-dotenv in /usr/local/lib/python3.6/dist-packages (0.15.0)


In [2]:
import os
import json
from pathlib import Path
from collections import OrderedDict

import requests
import psycopg2
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from IPython.display import clear_output

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/github/sir-protocol-bot/scraper'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/github/sir-protocol-bot/scraper


#### Scraper

In [4]:
class PensadorScraper:
  def __init__(self):
    self.links = OrderedDict()
    self.phrases = OrderedDict()
    self.next_links = ['/']
    self.previous_links = set()
    self.home = 'https://www.pensador.com'
  
  def get_links(self, url, soup):
    links = []
    for a in soup.find_all('a'):
      try:
        if a['href'].startswith('/') and \
           a['href'].find('.php') == -1 and \
           a['href'].find('/frase') == -1 and \
           a['href'].find('/colecao') == -1:
          links.append(a['href'])
      except:
        print('a tag without href property')
    links = set(links)
    self.links[url] = list(links)
    links = links - self.previous_links
    self.next_links.extend(links)

  def get_phrases(self, url, soup):
    for card in soup.find_all('div', 'thought-card'):
      try:
        id_ = card.find('p', 'frase')['id']
        phrase_ = card.find('p', 'frase').text
        phrase_ = phrase_.strip().replace('\u2060', '')
      except:
        continue
      
      try:
        author_ = card.find('span', 'autor').a.text
        author_url_ = card.find('span', 'autor').a['href']
      except:
        author_ = ''
        author_url_ = ''
      
      try:
        n_shares_ = card.find('div', class_='total-shares')
        n_shares_ = n_shares.text.replace(' compartilhamentos', '')
        if n_shares_[-4:] == ' mil':
          n_shares_ = n_shares_.replace(' mil', '')
          n_shares_ = float(n_shares_) * 1000
        elif n_shares_:
          n_shares_ = float(n_shares_)
        else:
          n_shares_ = 0
      except:
        n_shares_ = 0
        
      try:
        img_url_ = card['data-src']
      except:
        img_url_ = ''
      
      if id_ not in self.phrases:
        self.phrases[id_] = {
          'phrase': phrase_,
          'phrase_url': f'/frase/{id_}/',
          'author': author_,
          'author_url': author_url_,
          'n_shares': n_shares_,
          'img_url': img_url_,
          'urls': [url]
        }
      elif url not in self.phrases[id_]['urls']:
        self.phrases[id_]['urls'].append(url)
  
  def work(self):
    try:
      while self.next_links:
        url = self.next_links.pop()
        self.previous_links.add(url)

        try:
          page = requests.get(self.home + url)
          page.raise_for_status()
        except requests.exceptions.HTTPError as errh:
          print('HTTP Error\n', errh)
        except requests.exceptions.ConnectionError as errc:
          print('Error Connecting:\n', errc)
        except requests.exceptions.Timeout as errt:
          print('Timeout Error:\n', errt)
        except requests.exceptions.RequestException as err:
          print('An unexpected error:\n', err)
        else:
          soup = BeautifulSoup(page.content, 'html.parser')

          self.get_phrases(url, soup)
          self.get_links(url, soup)
          
          prev_size = len(self.previous_links)
          next_size = len(self.next_links)
          print(url, f'{prev_size}:{next_size}')
          clear_output(wait=True)
    except KeyboardInterrupt:
      pass
  
  def load(self):
    with open('./phrases.json', 'r') as file:
      self.phrases = json.load(file)
    with open('./links.json', 'r') as file:
      self.links = json.load(file)
    with open('./next_links.json', 'r') as file:
      self.next_links = json.load(file)
    with open('./previous_links.json', 'r') as file:
      self.previous_links = set(json.load(file))
  
  def save(self):
    with open('./phrases.json', 'w') as file:
      json.dump(self.phrases, file)
    with open('./links.json', 'w') as file:
      json.dump(self.links, file)
    with open('./next_links.json', 'w') as file:
      json.dump(self.next_links, file)
    with open('./previous_links.json', 'w') as file:
      json.dump(list(self.previous_links), file)
  
  def remove(self):
    Path('./phrases.json').unlink()
    Path('./links.json').unlink()
    Path('./next_links.json').unlink()
    Path('./previous_links.json').unlink()

In [5]:
scraper = PensadorScraper()

In [ ]:
scraper.work()

/mensagens_tocantes_de_agradecimento_profissional/ 571:6736


In [ ]:
scraper.save()

#### References

[1] www.postgresqltutorial.com/

[2] https://realpython.com/beautiful-soup-web-scraper-python/

[3] https://devcenter.heroku.com/articles/heroku-postgresql#connecting-in-python